###Задание 1.
Задание: обучите три классификатора:

на токенах с высокой частотой

на токенах со средней частотой

на токенах с низкой частотой

Сравните полученные результаты, оцените какие токены наиболее важные для классификации.

###Задание 2.
найти фичи с наибольшей значимостью, и вывести их

###Задание 3.
сравнить count/tf-idf/hashing векторайзеры/полносвязанную сетку (построить classification_report)

подобрать оптимальный размер для hashing векторайзера

убедиться что для сетки нет переобучения

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from nltk.tokenize import word_tokenize
from string import punctuation

from collections import Counter

from sklearn.linear_model import LogisticRegression # можно заменить на любимый классификатор
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings("ignore")

In [2]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = 1
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = 0
df = positive.append(negative)

In [3]:
df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",1
1,"Да, все-таки он немного похож на него. Но мой ...",1
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1


In [4]:
df.tail()

,text,label
111918,Но не каждый хочет что то исправлять:( http://...,0
111919,скучаю так :-( только @taaannyaaa вправляет мо...,0
111920,"Вот и в школу, в говно это идти уже надо(",0
111921,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",0
111922,Такси везет меня на работу. Раздумываю приплат...,0


In [5]:
x_train, x_test, y_train, y_test = train_test_split(df.text, df.label)

Посмотрим на качество с полным корпусом слов.

In [6]:
vec = CountVectorizer(ngram_range=(1, 1))
bow = vec.fit_transform(x_train) # bow -- bag of words (мешок слов)

In [7]:
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)

LogisticRegression(random_state=42)

In [8]:
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.76      0.76      0.76     27970
           1       0.77      0.77      0.77     28739

    accuracy                           0.77     56709
   macro avg       0.77      0.77      0.77     56709
weighted avg       0.77      0.77      0.77     56709



Соберем корпус из всех слов


In [9]:
# Setup
!pip install -q wordcloud
import wordcloud

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [10]:
corpus = [token for tweet in df.text for token in word_tokenize(tweet) if token not in punctuation]
print(len(corpus))

2870536


In [11]:
freq_dict = Counter(corpus)
freq_dict_sorted = sorted(freq_dict.items(), key=lambda x: -x[1])

Сортируем уникальные слова по их частоте

In [12]:
len(freq_dict_sorted)  # Всего уникальных слов

351101

Возьмем первые 10к популярных слов

In [13]:
first_try = freq_dict_sorted[10000:]

In [14]:
first_try = [x[0] for x in first_try]

In [15]:
vec = CountVectorizer(ngram_range=(1, 1), stop_words=first_try)
bow = vec.fit_transform(x_train) # bow -- bag of words (мешок слов)

In [16]:
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)

LogisticRegression(random_state=42)

In [17]:
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.73      0.74      0.74     27611
           1       0.75      0.75      0.75     29098

    accuracy                           0.74     56709
   macro avg       0.74      0.74      0.74     56709
weighted avg       0.74      0.74      0.74     56709



Возьмем 160к слов из середины частотного списка

In [18]:
second_try = freq_dict_sorted[:10000] + freq_dict_sorted[170000:]
second_try = [x[0] for x in second_try]

In [19]:
vec = CountVectorizer(ngram_range=(1, 1), stop_words=second_try)
bow = vec.fit_transform(x_train) # bow -- bag of words (мешок слов)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)

LogisticRegression(random_state=42)

In [20]:
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.75      0.63      0.69     33159
           1       0.58      0.71      0.64     23550

    accuracy                           0.66     56709
   macro avg       0.67      0.67      0.66     56709
weighted avg       0.68      0.66      0.67     56709



Возьмем последние 50к слов из частотного списка

In [21]:
last_try = freq_dict_sorted[:300000]
last_try = [x[0] for x in last_try]

In [22]:
vec = CountVectorizer(ngram_range=(1, 1), stop_words=last_try)
bow = vec.fit_transform(x_train) # bow -- bag of words (мешок слов)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)

LogisticRegression(random_state=42)

In [23]:
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.93      0.53      0.67     48730
           1       0.21      0.75      0.32      7979

    accuracy                           0.56     56709
   macro avg       0.57      0.64      0.50     56709
weighted avg       0.83      0.56      0.63     56709



Из 350000 уникальных слов взял первые 10000, что показало лучший результат. Т.е. самые популярные - самые важные.

In [24]:
vec = CountVectorizer(ngram_range=(1, 1))
bow = vec.fit_transform(x_train) # bow -- bag of words (мешок слов)

In [25]:
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)

LogisticRegression(random_state=42)

In [26]:
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.76      0.76      0.76     27970
           1       0.77      0.77      0.77     28739

    accuracy                           0.77     56709
   macro avg       0.77      0.77      0.77     56709
weighted avg       0.77      0.77      0.77     56709



In [27]:
list_feat_names = [x[0] for x in list(vec.vocabulary_.items())]

In [28]:
features = [x for x in zip(clf.coef_[0], list_feat_names)]

In [29]:
sorted_features = sorted(features, key=lambda x: -abs(x[0]))

Отсортированные фичи по весам (по модулю)

In [30]:
sorted_features[:50]

[(-9.797165795537651, '9hcruqsmtw'),
 (-7.306505074899978, 'матведу'),
 (-7.061771474010772, 'n_grigoryan'),
 (6.511646278717665, 'флешмобе'),
 (-6.2943294831758, 'опп'),
 (-6.168774144192902, 'шивы'),
 (5.369609395203617, '_solembum'),
 (5.250932245624115, 'огоньки'),
 (-4.982046098777121, 'приятном'),
 (-4.91311250149462, 'фг'),
 (4.908088695040037, 'капитолии'),
 (-4.8933046077113636, 'извинятся'),
 (-4.581464409883485, 'утрам'),
 (-4.477252917706959, 'ппу'),
 (-4.448166076970339, 'kra_sta'),
 (-4.42217364901098, 'deniskorzun'),
 (-4.091910350129765, 'девки'),
 (-3.9091155158412767, 'аццки'),
 (-3.8991905025301086, 'ungatoenzapatos'),
 (-3.817954210928509, 'умеющего'),
 (3.620240192348078, 'чудному'),
 (-3.619684825633385, 'заявлять'),
 (3.465961244747148, 'dashanovitskaya'),
 (-3.413233194181286, 'принципиальное'),
 (-3.348962720343475, 'поправка'),
 (-3.1679643891082554, 'подкона'),
 (-3.1553762084858983, 'qlpbvq5hwg'),
 (-3.1331632135917054, 'судоку'),
 (3.123262503848965, 'ikopo

Потестим разные эмбединги
CountV

In [31]:
vec = CountVectorizer(ngram_range=(1, 1))
bow = vec.fit_transform(x_train) # bow -- bag of words (мешок слов)

In [32]:
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)

LogisticRegression(random_state=42)

In [33]:
pred = clf.predict(vec.transform(x_train))
print(classification_report(pred, y_train))

              precision    recall  f1-score   support

           0       0.90      0.90      0.90     83879
           1       0.91      0.90      0.90     86246

    accuracy                           0.90    170125
   macro avg       0.90      0.90      0.90    170125
weighted avg       0.90      0.90      0.90    170125



In [34]:
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)

LogisticRegression(random_state=42)

In [36]:
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.76      0.76      0.76     27970
           1       0.77      0.77      0.77     28739

    accuracy                           0.77     56709
   macro avg       0.77      0.77      0.77     56709
weighted avg       0.77      0.77      0.77     56709



Tf-Idf

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [38]:
vec = TfidfVectorizer(ngram_range=(1, 1))
bow = vec.fit_transform(x_train) # bow -- bag of words (мешок слов)

In [39]:
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)

LogisticRegression(random_state=42)

In [40]:
pred = clf.predict(vec.transform(x_train))
print(classification_report(pred, y_train))

              precision    recall  f1-score   support

           0       0.83      0.86      0.84     80988
           1       0.87      0.84      0.85     89137

    accuracy                           0.85    170125
   macro avg       0.85      0.85      0.85    170125
weighted avg       0.85      0.85      0.85    170125



In [41]:
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.74      0.77      0.75     26714
           1       0.79      0.76      0.77     29995

    accuracy                           0.76     56709
   macro avg       0.76      0.76      0.76     56709
weighted avg       0.76      0.76      0.76     56709



Hashing
*подобрал оптимально количество фичей - 10к, при увеличении в дальнейшем прирост качества не значительный, относительно количества увеличенных фичей.

In [42]:
from sklearn.feature_extraction.text import HashingVectorizer

In [43]:
vec = HashingVectorizer(n_features=10000)
bow = vec.fit_transform(x_train) # bow -- bag of words (мешок слов)

In [44]:
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)

LogisticRegression(random_state=42)

In [45]:
pred = clf.predict(vec.transform(x_train))
print(classification_report(pred, y_train))

              precision    recall  f1-score   support

           0       0.73      0.75      0.74     81861
           1       0.76      0.74      0.75     88264

    accuracy                           0.74    170125
   macro avg       0.74      0.74      0.74    170125
weighted avg       0.74      0.74      0.74    170125



In [46]:
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)

LogisticRegression(random_state=42)

In [47]:
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.70      0.72      0.71     27166
           1       0.73      0.72      0.72     29543

    accuracy                           0.72     56709
   macro avg       0.72      0.72      0.72     56709
weighted avg       0.72      0.72      0.72     56709



TF-IDF показал себя лучше остальных, однако Hashing позволил сократить размерность, что сильно повлияло на скорость. Также из всех моделей при использовании Hasing обобщающая способность модели лучше всего, модель почти не переобучилась.